In [1]:
import cobra
from cobra import Reaction
import pandas as pd
pd.set_option('display.max_colwidth', -1)
import ast

In [12]:
M = cobra.io.load_json_model('Recon3D_301/Recon3DModel_301_simple_medium.json')

M.reactions.GAPD.subsystem

['Taurine and hypotaurine metabolism',
 'Transport, mitochondrial',
 'Ubiquinone synthesis',
 'Tyrosine metabolism']

# Save JSON model with pythonified IDs and subsystem information

In [2]:
M = cobra.io.load_matlab_model('Recon3D_301/Recon3DModel_301.mat') # takes several minutes

# fix reaction and metabolite IDs
for rxn in M.reactions:
    rxn.id = rxn.id.replace('[','_').replace(']','')

M.repair()

for m in M.metabolites:
    m.id = m.id.replace('[','_').replace(']','')

M.repair()

# add ATPM reaction
ATPM = Reaction('ATPM')
M.add_reaction(ATPM)
M.reactions.ATPM.lower_bound = 0
M.reactions.ATPM.add_metabolites({'atp_c':-1.0,'adp_c':1.0,'pi_c':1.0})
M.repair()

# set objective to full biomass
M.reactions.biomass_maintenance.objective_coefficient = 0
M.reactions.biomass_reaction.objective_coefficient = 1

# Add subsystem information

rxn_ss = pd.read_csv('./Recon3D_301/subsystem_data/Recon3_rxn_subsystem.csv',index_col=0).set_index('rxn ID')

for rxn in rxn_ss.index:
    rxn_updated_id = rxn.replace('[','_').replace(']','')
    
    ss = ast.literal_eval(rxn_ss.loc[rxn]['subSystem'])
    
    M.reactions.get_by_id(rxn_updated_id).subsystem = ss
    
# save
cobra.io.save_json_model(M, 'Recon3D_301/Recon3DModel_301_simple_medium.json')

M.reactions.RADH.subsystem

['Transport, extracellular',
 'Transport, mitochondrial',
 'Tyrosine metabolism',
 'Methionine and cysteine metabolism']

# Reduce medium + exits
Set up a simple medium of glucose, oxygen, essential amino acids and a couple of other metabolites. Block all sinks and DM reactions. 

In [4]:
model = M.copy()

# Block sink and demand reactions
for rxn in model.reactions:
    if 'sink_' in rxn.id or 'DM_' in rxn.id:
        rxn.upper_bound = 0
        rxn.lower_bound = 0
        
# set very simple medium that allows growth
for rxn in model.reactions:
    if 'EX_' in rxn.id:
        rxn.upper_bound = 0 # block all outward
        rxn.lower_bound = 0
        
for rxn in ['EX_h2o_e','EX_h_e','EX_co2_e','EX_nh4_e','EX_pi_e','EX_hco3_e','EX_so4_e','EX_o2_e']:
    model.reactions.get_by_id(rxn).lower_bound = - 999999

for rxn in ['EX_his_L_e','EX_ile_L_e','EX_leu_L_e','EX_lys_L_e','EX_met_L_e','EX_phe_L_e','EX_thr_L_e','EX_trp_L_e','EX_val_L_e',
            'EX_ps_hs_e', # phsophatidylserine
           ]:
    model.reactions.get_by_id(rxn).lower_bound = -0.1
    
model.reactions.EX_glc_D_e.lower_bound = -1
        
for rxn in ['EX_h2o_e','EX_h_e','EX_nh4_e','EX_pi_e','EX_so4_e',
           'EX_co2_e','EX_lac_D_e','EX_lac_L_e',
           'EX_urea_e','EX_acac_e','EX_bhb_e','EX_ppa_e','EX_cyst_L_e' ]: # 
    model.reactions.get_by_id(rxn).upper_bound = 1000
        
sol = cobra.flux_analysis.pfba(model)
print('After changes',sol.fluxes['biomass_reaction'])
model.summary()

After changes 0.16888639687627705
IN FLUXES        OUT FLUXES     OBJECTIVES
---------------  -------------  ----------------------
o2_e     2.37    h2o_e  2.82    biomass_reac...  0.169
glc_D_e  0.517   co2_e  1.9
nh4_e    0.464   ppa_e  0.125
pi_e     0.162   so4_e  0.0663
his_L_e  0.1
leu_L_e  0.1
lys_L_e  0.1
met_L_e  0.1
phe_L_e  0.1
val_L_e  0.1
ile_L_e  0.0996
trp_L_e  0.0652
thr_L_e  0.0528
ps_hs_e  0.0477


# Save the model

In [5]:
cobra.io.save_json_model(model, 'Recon3D_301/Recon3DModel_301_simple_medium.json')

M = cobra.io.load_json_model('Recon3D_301/Recon3DModel_301_simple_medium.json')
model = M.copy()

# Check oxygen dependence

In [6]:
model.reactions.EX_o2_e.lower_bound = 0
print(model.optimize())

# model.metabolites.get_by_id('o2_c').summary()

<Solution 0.000 at 0x11eec7fd0>


## How is ATP generated? 
### When maximizing ATPM using only glucose and o2

In [7]:
model = M.copy()

for rxn in ['EX_his_L_e','EX_ile_L_e','EX_leu_L_e','EX_lys_L_e','EX_met_L_e','EX_phe_L_e','EX_thr_L_e','EX_trp_L_e','EX_val_L_e']:
    model.reactions.get_by_id(rxn).lower_bound = 0

model.objective = model.reactions.ATPM
model.optimize()

model.summary()
print()
model.metabolites.atp_c.summary()
print()
model.metabolites.atp_m.summary()

IN FLUXES       OUT FLUXES    OBJECTIVES
--------------  ------------  ------------
h_e      31.6   h2o_e  37.8   ATPM  32
o2_e      5.91  co2_e   6
glc_D_e   1

PRODUCING REACTIONS -- Adenosine Triphosphate (atp_c)
-----------------------------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  ---------------------------------------
94%    58.5  ATPtm     adp_c + atp_m --> adp_m + atp_c
3%      2    PGK       3pg_c + atp_c <=> 13dpg_c + adp_c
3%      2    PYK       adp_c + h_c + pep_c --> atp_c + pyr_c

CONSUMING REACTIONS -- Adenosine Triphosphate (atp_c)
-----------------------------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  ---------------------------------------
51%    32    ATPM      atp_c --> adp_c + pi_c
46%    28.5  DURIK1    atp_c + duri_c --> adp_c + dump_c + h_c
2%      1    HEX7      atp_c + fru_c --> adp_c + f6p_c + h_c
2%      1    PFK       atp_c + f6p_c --> adp_c + fdp_c + h_c

PRODUCING REACTIONS -- Ade

### When maximizing biomass

In [8]:
model = M.copy()

model.optimize()

model.summary()
print()
model.metabolites.atp_c.summary()
print()
model.metabolites.atp_m.summary()

IN FLUXES        OUT FLUXES      OBJECTIVES
---------------  --------------  ----------------------
o2_e     3.22    h2o_e   4.31    biomass_reac...  0.169
glc_D_e  1       co2_e   2.74
nh4_e    0.436   acac_e  0.55
pi_e     0.162   ppa_e   0.102
his_L_e  0.1     so4_e   0.0535
leu_L_e  0.1
lys_L_e  0.1
phe_L_e  0.1
thr_L_e  0.1
val_L_e  0.1
trp_L_e  0.0876
met_L_e  0.0872
ile_L_e  0.0483
ps_hs_e  0.0477
hco3_e   0.0194

PRODUCING REACTIONS -- Adenosine Triphosphate (atp_c)
-----------------------------------------------------
%      FLUX  RXN ID      REACTION
---  ------  ----------  --------------------------------------------------
52%  3.84    ATPtm       adp_c + atp_m --> adp_m + atp_c
25%  1.83    PGK         3pg_c + atp_c <=> 13dpg_c + adp_c
23%  1.71    PYK         adp_c + h_c + pep_c --> atp_c + pyr_c

CONSUMING REACTIONS -- Adenosine Triphosphate (atp_c)
-----------------------------------------------------
%      FLUX  RXN ID      REACTION
---  ------  ----------  ----------

## How does the model behave under oxygen limitation? 

In [9]:
model = M.copy()

model.reactions.EX_o2_e.lower_bound = -0.1
sol = cobra.flux_analysis.pfba(model)

model.summary()
print()
model.metabolites.atp_c.summary()
print()
model.metabolites.atp_m.summary()

IN FLUXES         OUT FLUXES        OBJECTIVES
----------------  ----------------  -----------------------
glc_D_e  1        h_e      1.81     biomass_reac...  0.0787
nh4_e    0.207    lac_L_e  1.55
his_L_e  0.1      h2o_e    0.427
o2_e     0.1      co2_e    0.104
pi_e     0.0762   acac_e   0.0695
lys_L_e  0.0466   ppa_e    0.00367
leu_L_e  0.0429
phe_L_e  0.033
val_L_e  0.0278
thr_L_e  0.0246
ile_L_e  0.0225
ps_hs_e  0.0216
met_L_e  0.0157
trp_L_e  0.00105

PRODUCING REACTIONS -- Adenosine Triphosphate (atp_c)
-----------------------------------------------------
%      FLUX  RXN ID      REACTION
---  ------  ----------  --------------------------------------------------
48%  1.92    PGK         3pg_c + atp_c <=> 13dpg_c + adp_c
47%  1.87    PYK         adp_c + h_c + pep_c --> atp_c + pyr_c
5%   0.221   ATPtm       adp_c + atp_m --> adp_m + atp_c

CONSUMING REACTIONS -- Adenosine Triphosphate (atp_c)
-----------------------------------------------------
%      FLUX  RXN ID      REACTI